# plot the QC filter results for satellite DA (cris-fsr
Adapted from Haidao Lin's script

## get the `pyDAmonitor_ROOT` env variable
This step is highly recommended. It is required if one want to use the DAmonitor Python package or use the MPAS/FV3 sample data or local cartopy nature_earth_data.

In [ ]:
%%time
# autoload external python modules if they changed
%load_ext autoreload
%autoreload 2
    
import sys, os
pyDAmonitor_ROOT=os.getenv("pyDAmonitor_ROOT")
if pyDAmonitor_ROOT is None:
    print("!!! pyDAmonitor_ROOT is NOT set. Run `source ush/load_pyDAmonitor.sh`")
else:
    print(f"pyDAmonitor_ROOT={pyDAmonitor_ROOT}\n")
sys.path.insert(0, pyDAmonitor_ROOT)

## import modules

In [ ]:
%%time
from netCDF4 import Dataset
import cartopy
cartopy.config['data_dir'] = f"{pyDAmonitor_ROOT}/data/natural_earth_data"
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
import numpy as np
#matplotlib.use("Agg")
%matplotlib inline

from DAmonitor.base import query_data, query_dataset, query_obj, to_dataframe
from DAmonitor.obs import obsSpace

## read satellite diagnostics using the `obsSpace` class

In [ ]:
%%time
obsfile = os.path.join(pyDAmonitor_ROOT,'data/mpasjedi/jdiag_cris-fsr_n20.nc')
obs = obsSpace(obsfile)

## check attributes, dimensions and the 'bt' data dictionary

In [ ]:
for attr in obs.ds.ncattrs():
    print(f'{attr}: {obs.ds.getncattr(attr)}')

obs.ds.dimensions

In [ ]:
query_data(obs.bt, meta_exclude="sensorCentralWavenumber_")

In [ ]:
obs.bt.CloudDetectMinResidualIR.compressed()

## find channels with valid ombg masked by CloudDetectMinResidualIR==1

In [ ]:
MAX_LINES = 20
knt = 0
for iChan, chanID in enumerate(obs.bt.Channel):
    mask = obs.bt.CloudDetectMinResidualIR[:, iChan] == 1
    if obs.bt.ombg[mask, iChan].size > 0:
        knt += 1
        if knt <= MAX_LINES:
            print(iChan, chanID, obs.bt.ombg[mask, iChan].size)  #  250 648 1083

## Define the sat_obs_map(..) function

In [ ]:
def sat_obs_map(obs,iChan, area, colors, qcfilters):
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))
    
    # Plot grid lines
    # ----------------
    gl = ax.gridlines(crs=ccrs.PlateCarree(central_longitude=0), draw_labels=True,
                      linewidth=1, color='gray', alpha=0.5, linestyle='-')
    gl.top_labels = False
    gl.xlabel_style = {'size': 5, 'color': 'black'}
    gl.ylabel_style = {'size': 5, 'color': 'black'}
    # gl.xlocator = mticker.FixedLocator(
        # [-180, -135, -90, -45, 0, 45, 90, 135, 179.9])

    qcfilterList = qcfilters.split(',')
    colorList = colors.split(',')

    for qcfilter, color in zip(qcfilterList, colorList):
        mask = obs.bt[qcfilter][:, iChan] == 1
        field = obs.bt.ombg[mask, iChan]
        lon = obs.bt.longitude[mask]
        lat = obs.bt.latitude[mask]
        knt = np.ma.count(field)
        sc = ax.scatter(lon, lat, color=color, 
                        label=f"{qcfilter}, Total Count: {knt}",
                        s=2, linewidth=0, transform=ccrs.PlateCarree(), norm=None, antialiased=True)
        
    ax.set_extent(area)
    ax.coastlines()
    plt.title(f"{qcfilter}, Total Count: {knt}")
    plt.show()

In [ ]:
iChan = 250
area = [-150, -50, 15, 55]

#
colors = "blue"
qcfilters = "CloudDetectMinResidualIR"
sat_obs_map(obs, iChan, area, colors, qcfilters)
#
colors = "blue"
qcfilters = "GrossCheck"
sat_obs_map(obs, iChan, area, colors, qcfilters)
#
# plot CloudDetectMinResidualIR and GrossCheck together
colors = "blue,red"
qcfilters = "CloudDetectMinResidualIR,GrossCheck"
qcfilters.split(',')
sat_obs_map(obs, iChan, area, colors, qcfilters)
#
colors = "blue"
qcfilters = "NearSSTRetCheckIR"
sat_obs_map(obs, iChan, area, colors, qcfilters)
#
colors = "blue"
qcfilters = "UseflagCheck"
sat_obs_map(obs, iChan, area, colors, qcfilters)

## Use a slidebar to quickly navigate through all channels

In [ ]:
%%time
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'

# 
qcfilter = "CloudDetectMinResidualIR"

data = {}
for iChan, ch in enumerate(obs.bt.Channel):
    mask = obs.bt[qcfilter][:, iChan] == 1
    field = obs.bt.ombg[mask, iChan]
    lon = obs.bt.longitude[mask]
    lat = obs.bt.latitude[mask]
    knt = np.ma.count(field)
    data[f"{ch}"] = {
        "lat": lat,
        "lon": lon,
        "knt": knt,
    }

fig = go.Figure()

# Add initial trace (first channel)
ch0 = f"{obs.bt.Channel[0]}"
fig.add_trace(go.Scattermap(
    lat=data[ch0]["lat"],
    lon=data[ch0]["lon"],
    mode='markers',
    marker=dict(size=10, color='blue'),
    name=ch0
))

# Define slider steps to replace data
steps = []
for ch in data:
    step = dict(
        method="update",
        args=[{"lat": [data[ch]["lat"]], "lon": [data[ch]["lon"]], "name": ch},
              {"title.text": f"Channel: {ch}"},],
        label=ch
    )
    steps.append(step)

sliders = [dict(active=0, pad={"t": 50}, steps=steps)]

# Layout
fig.update_layout(
    map=dict(
        center={"lat": 36.5, "lon": -118},  # adjust for your data
        zoom=4
    ),
    sliders=sliders,
    width=1000,   # pixels
    height=800,   # pixels
)

fig.show()

In [ ]:
query_dataset(obs.ds, meta_exclude="sensorCentralWavenumber")